# $T_s$ from $P_{\{R,G,B\}}$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
import os
import datetime
import tensorflow.keras.backend as K

########## CONFIGURATION ##########
R_PATH = './npy-rgb-ts/R.npy'
G_PATH = './npy-rgb-ts/G.npy'
B_PATH = './npy-rgb-ts/B.npy'

TS_PATH = './npy-rgb-ts/ts.npy'
FS_PATH = './npy-rgb-ts/fs.npy'
np.random.seed(10) #10
tf.random.set_seed(10) #10
 
NOISE_PERCENTAGE =  1
############################

TITLE = 'rgb-ts'
MODEL_H5_PATH = f'./models/{TITLE}.h5'

def standardize(data, mean, std):
    return (data - mean) / std

def destandardize(data, mean, std):
    return (data * std) + mean

def mae_destandardize(y_true, y_pred):
    y_true_d = destandardize(y_true, y_mean, y_std)
    y_pred_d = destandardize(y_pred, y_mean, y_std)
    return K.mean(K.abs(y_pred_d - y_true_d))

def mae_percentage(y_true, y_pred):
    y_true_d = destandardize(y_true, y_mean, y_std)
    y_pred_d = destandardize(y_pred, y_mean, y_std)
    mae = K.mean(K.abs(y_pred_d - y_true_d))
    return (mae * 100) / y_mean

def predict_single(data, x_mean, x_std):
    data_map = standardize(data, x_mean, x_std)
    pred = model.predict(np.expand_dims(data_map, axis=0))[0]
    return destandardize(pred, y_mean, y_std)

def predict(data, x_mean, x_std):
    assert data.ndim > 1
    data_s = standardize(data, x_mean, x_std)
    if data_s.ndim == 2:
        data_s = np.expand_dims(data_s, axis=0)
        pred = model.predict(data_s)[0]
    else:
        pred = model.predict(data_s)
    return destandardize(pred, y_mean, y_std)


def add_noise(data, rng, noise_stddev):
    assert np.size(noise_stddev) == 1 or np.size(noise_stddev) == 2
    if np.size(noise_stddev) == 1:
        data += rng.normal(0, noise_stddev, data.shape)
    else:
        s = rng.uniform(noise_stddev[0], noise_stddev[1], len(data))
        for i, d in enumerate(data):
            d += rng.normal(0, s[i], d.shape)
    return data

def format_to_train(x1,x2,x3):
    x1_t = np.transpose(x1, [0,3,1,2])
    x2_t = np.transpose(x2, [0,3,1,2])
    x3_t = np.transpose(x3, [0,3,1,2])
    xd = np.concatenate((x1_t,x2_t,x3_t), axis=1)
    xd = np.transpose(xd, [0,2,3,1])
    return xd

def max_mean(B1):
    B_max = list()
    for i in range(len(B1)):
        B_max.append(B1[i,:,:].max())
    B_max_mean = np.array(B_max).mean()
    print('max_mean: ',B_max_mean)
    return B_max_mean
    
def destandardize_py(x_train):
    d1 = x_train[:,:,0]
    d1 = destandardize(d1, x1_mean,x1_std)
    d2 = x_train[:,:,1]
    d2 = destandardize(d2, x2_mean,x2_std)
    d3 = x_train[:,:,2]
    d3 = destandardize(d3, x3_mean,x3_std)
    d1 = d1.reshape(d1.shape[0],d1.shape[1], 1)
    d2 = d2.reshape(d2.shape[0],d2.shape[1], 1)
    d3 = d3.reshape(d3.shape[0],d3.shape[1], 1)
    d = np.concatenate((d1,d2,d3), axis =2)
    return d


## Training

In [ ]:
x1 = np.load(R_PATH)
x2 = np.load(G_PATH)
x3 = np.load(B_PATH)
y = np.load(TS_PATH)


##  152 x 64;  7.6cm x 0.64cm
offset = 128
x1 = x1[:,:offset,:40] #126....#7.6cm x 0.4cm
x2 = x2[:,:offset,:40]
x3 = x3[:,:offset,:40]
y  =  y[:,:offset,:40]

# Normalization
for i in range(len(x1)):
    x_max = np.max([x1[i].max(),x2[i].max(),x3[i].max()])
    x2[i] = x2[i]/x_max
    x3[i] = x3[i]/x_max
    x1[i] = x1[i]/x_max


x1_mean = np.mean(x1[:].mean())
x1_std = np.mean(x1[:].std())

x2_mean = np.mean(x2[:].mean())
x2_std = np.mean(x2[:].std())

x3_mean = np.mean(x3[:].mean())
x3_std = np.mean(x3[:].std())

y_mean = np.mean(y[:].mean())
y_std = np.mean(y[:].std())

x3_max_mean = np.mean([x3[i,:,:].max() for i in range(len(x3))])

#### NOISE
if NOISE_PERCENTAGE > 0:
    noise_stddev = [0, x3_max_mean*(NOISE_PERCENTAGE/100)]
    x1 = add_noise(x1, np.random.RandomState(0), noise_stddev)
    x2 = add_noise(x2, np.random.RandomState(1), noise_stddev)
    x3 = add_noise(x3, np.random.RandomState(2), noise_stddev)
####

x1 = np.expand_dims(x1, axis=3)
x2 = np.expand_dims(x2, axis=3)
x3 = np.expand_dims(x3, axis=3)

x = format_to_train(x1,x2,x3)

# Train set / Validatio set split
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, shuffle=True)

# Standardization
x_train[:,:,:,0] = standardize(x_train[:,:,:,0], x1_mean, x1_std)
x_valid[:,:,:,0] = standardize(x_valid[:,:,:,0], x1_mean, x1_std)
x_train[:,:,:,1] = standardize(x_train[:,:,:,1], x2_mean, x2_std)
x_valid[:,:,:,1] = standardize(x_valid[:,:,:,1], x2_mean, x2_std)
x_train[:,:,:,2] = standardize(x_train[:,:,:,2], x3_mean, x3_std)
x_valid[:,:,:,2] = standardize(x_valid[:,:,:,2], x3_mean, x3_std)

y_train = standardize(y_train, y_mean, y_std)
y_valid = standardize(y_valid, y_mean, y_std)

## U-net

In [ ]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, BatchNormalization, Activation, Dense, Dropout,GlobalAveragePooling2D
from tensorflow.keras.layers import Lambda, RepeatVector, Reshape
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D, GlobalMaxPool2D
from tensorflow.keras.layers import concatenate, add, subtract
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

def conv_block(tensor, filters, kernel_size=3):
    c = Conv2D(
        filters=filters,
        kernel_size=kernel_size,
        activation = 'relu',
        kernel_initializer='he_normal',
        padding='same'
    )(tensor)
    c = BatchNormalization()(c)
    c = Conv2D(
        filters=filters,
        kernel_size=kernel_size,
        activation = 'relu',
        kernel_initializer='he_normal',
        padding='same'
    )(c)
    c = BatchNormalization()(c)
    return c

def deconv_block(tensor, filters, kernel_size=3):
    d = Conv2DTranspose(
        filters=filters,
        kernel_size=kernel_size,
        strides=(2,2),
        activation='relu',
        kernel_initializer='he_normal',
        padding='same'
    )(tensor)
    d = BatchNormalization()(d)
    return d

    
def model_unet(input_img, depth=4, first_filters=64, kernel_size=3, dropout=None, skip_connection=False):
    # Contracting Path
    c = []
    for i in range(depth):
        if i == 0:
            c.append(conv_block(input_img, filters=(2**i)*first_filters))
        else:
            c.append(MaxPooling2D(pool_size=2)(c[i-1]))
            c[i] = conv_block(c[i], filters=(2**i)*first_filters)
        if dropout is not None:
            c[i] = Dropout(dropout)(c[i])
    # Expanding Path
    e = c[-1]
    for i in reversed(range(depth-1)):
        e = deconv_block(e, filters=(2**i)*first_filters)
        e = concatenate([e, c[i]])
        e = conv_block(e, filters=(2**i)*first_filters)
        if dropout is not None:
            c[i] = Dropout(dropout)(c[i])
        if i == 0:
            e = Conv2D(filters=1, kernel_size=1, activation='linear') (e)
    if skip_connection:
        e = add([input_img, e])
    model = Model(inputs=[input_img], outputs=[e])
    return model


inputs = Input(shape=(x_train.shape[1],x_train.shape[2],3))
model = model_unet(inputs, depth=4, first_filters=8, dropout=None, skip_connection=False)


model.compile(
    #optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
    optimizer= Adam(lr = 0.0005),
    loss='mse',
    metrics=['mae', mae_destandardize, mae_percentage]
)

es_cb = EarlyStopping(monitor='val_mae_percentage', mode='min', verbose=2, patience=10)
cp_cb = ModelCheckpoint(MODEL_H5_PATH, monitor='val_mae_percentage', verbose=1, save_best_only=True, mode='min')

history = model.fit(x = x_train, y = y_train,
        epochs = 100,
        #batch_size = 128,
        validation_data = (x_valid, y_valid),
        callbacks = [es_cb, cp_cb])

model = tf.keras.models.load_model(
    MODEL_H5_PATH,
    custom_objects={'mae_destandardize': mae_destandardize, 'mae_percentage': mae_percentage}
)

## Model summary

In [ ]:
model.summary()

## Inference over the test set

In [ ]:
x1_test = np.load(R_PATH.replace('.npy', '_test.npy'))
x2_test = np.load(G_PATH.replace('.npy', '_test.npy'))
x3_test = np.load(B_PATH.replace('.npy', '_test.npy'))
y_test = np.load(TS_PATH.replace('.npy', '_test.npy'))
fs_test = np.load(FS_PATH.replace('.npy', '_test.npy'))

off_set = 128
x1_test = x1_test[:,:off_set,:40]
x2_test = x2_test[:,:off_set,:40]
x3_test = x3_test[:,:off_set,:40]
fs_test = fs_test[:,:off_set,:40]
y_test = y_test[:,:off_set,:40]

for i in range(len(x1_test)):
    x_max = np.max([x1_test[i].max(),x2_test[i].max(),x3_test[i].max()])
    x1_test[i] = x1_test[i]/x_max
    x2_test[i] = x2_test[i]/x_max
    x3_test[i] = x3_test[i]/x_max

x3_max_mean = np.mean([x3_test[i,:,:].max() for i in range(len(x3_test))])

#### NOISE
TEST_NOISE_PERCENTAGE = 0.5
if TEST_NOISE_PERCENTAGE > 0:
    noise_stddev = [0, x3_max_mean*(TEST_NOISE_PERCENTAGE/100)]
    x1_test = add_noise(x1_test, np.random.RandomState(0), noise_stddev)
    x2_test = add_noise(x2_test, np.random.RandomState(1), noise_stddev)
    x3_test = add_noise(x3_test, np.random.RandomState(2), noise_stddev)
####

x1_test[x1_test<0] = 0
x2_test[x2_test<0] = 0
x3_test[x3_test<0] = 0 
    
x1_test = np.expand_dims(x1_test, axis=3)
x2_test = np.expand_dims(x2_test, axis=3)
x3_test = np.expand_dims(x3_test, axis=3)

x_test = format_to_train(x1_test,x2_test,x3_test)

x_test[:,:,:,0] = standardize(x_test[:,:,:,0], x1_mean, x1_std)
x_test[:,:,:,1] = standardize(x_test[:,:,:,1], x2_mean, x2_std)
x_test[:,:,:,2] = standardize(x_test[:,:,:,2], x3_mean, x3_std)

# Loading the model
model = tf.keras.models.load_model(
    MODEL_H5_PATH,
    custom_objects={'mae_destandardize': mae_destandardize, 'mae_percentage': mae_percentage}
)
y_test_pred = model.predict(x_test)
y_test_pred = destandardize(y_test_pred, y_mean, y_std)[:,:,:,0]

# Keeping only the area were there is a significant concentration of soot
y_test[fs_test <= 0.1e-6] = 0
y_test_pred[fs_test <= 0.1e-6] = 0


## Plot

In [ ]:
# CONFIGURATION

N = 1

# ==========

import matplotlib.ticker as ticker
XTICKS = 2
YTICKS = 8
%matplotlib inline
plt.rcParams['figure.figsize'] = [14, 12]

fig, ax = plt.subplots(2, 4)

def aximshow(ax, data, title=None, cmap=None, vmin=None, vmax=None):
    im = ax.imshow(data, cmap=cmap, vmin=vmin, vmax=vmax) 
    ax.xaxis.set_major_locator(
        ticker.FixedLocator(np.linspace(0, data.shape[1]-1, XTICKS)))
    ax.set_xticklabels(np.around(np.linspace(0, 0.4, XTICKS), 1))
    ax.yaxis.set_major_locator(
        ticker.FixedLocator(np.linspace(0, data.shape[0]-1, YTICKS)))
    ax.set_yticklabels(np.around(np.linspace(7.6, 1.3, YTICKS), 1))
    ax.set_xlabel(r'$r$ (cm)')
    ax.set_ylabel(r'$z$ (cm)')
    ax.set_title(title)
    return im

def remove_tick_labels(ax):
    ax.tick_params(labelleft=False, labelbottom=False)
    ax.set_xlabel('')
    ax.set_ylabel('')

x1_test = np.load(R_PATH.replace('.npy', '_test.npy'))
x2_test = np.load(G_PATH.replace('.npy', '_test.npy'))
x3_test = np.load(B_PATH.replace('.npy', '_test.npy'))
off_set = 128
x1_test = x1_test[:,:off_set,:40]
x2_test = x2_test[:,:off_set,:40]
x3_test = x3_test[:,:off_set,:40]
for i in range(len(x1_test)):
    x_max = np.max([x1_test[i].max(),x2_test[i].max(),x3_test[i].max()])
    x1_test[i] = x1_test[i]/x_max
    x2_test[i] = x2_test[i]/x_max
    x3_test[i] = x3_test[i]/x_max

aximshow(ax[0,0], x1_test[N], title=r'$P_R$', cmap='jet', vmin=x3_test[N].min(), vmax=x1_test[N].max())
aximshow(ax[0,1], x2_test[N], title=r'$P_G$', cmap='jet', vmin=x3_test[N].min(), vmax=x1_test[N].max())
aximshow(ax[0,2], x3_test[N], title=r'$P_B$', cmap='jet', vmin=x3_test[N].min(), vmax=x1_test[N].max())
remove_tick_labels(ax[0,1])
remove_tick_labels(ax[0,2])
ax[0,3].set_axis_off()

# ==========

vmin = 1500
vmax = y_test[N].max()

abs_err = np.abs(y_test[N] - y_test_pred[N])

im = aximshow(ax[1,0], y_test[N], title=r'$T_s$ Groundtruth', cmap='jet', vmin=vmin, vmax=vmax)
fig.colorbar(im, ax=ax[1,0], label=r'$T_s$ (K)')
im = aximshow(ax[1,1], y_test_pred[N], title=r'$T_s$ Prediction', cmap='jet', vmin=vmin, vmax=vmax)
fig.colorbar(im, ax=ax[1,1], label=r'$T_s$ (K)')
im = aximshow(ax[1,2], abs_err, title=r'$T_s$ Abs. Error', cmap='jet', vmin=vmin, vmax=vmax)
fig.colorbar(im, ax=ax[1,2], label=r'$T_s$ (K)')
im = aximshow(ax[1,3], abs_err, title=r'$T_s$ Abs. Error' + '\n(Different Scale)', cmap='jet')
fig.colorbar(im, ax=ax[1,3], label=r'$T_s$ (K)')
remove_tick_labels(ax[1,1])
remove_tick_labels(ax[1,2])
remove_tick_labels(ax[1,3])

print(f'Image {N}, Abs. error max:\t {abs_err.max()}')
print(f'Image {N}, Abs. error min:\t {abs_err.min()}')
print(f'Image {N}, Abs. error mean:\t {abs_err.mean()}')
print(f'Image {N}, Abs. error stddev:\t {abs_err.std()}')
print(f'Image {N}, Abs. error %:\t\t {abs_err.mean()*100/y_test[N].mean()}')